In [196]:
import torch
import pandas as pd 
import numpy as np 
from torch.utils.data import DataLoader,Dataset
from torchinfo import summary
import torch.nn as nn 
from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [197]:
df = pd.read_csv('riceClassification.csv')
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [198]:
df.drop('id',inplace=True,axis=1)
df.set_index('Area',inplace=True)
df.head()


,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
Area,,,,,,,,,,
4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [199]:
df.shape

(18185, 10)

In [200]:
df.columns

Index(['MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'ConvexArea',
       'EquivDiameter', 'Extent', 'Perimeter', 'Roundness', 'AspectRation',
       'Class'],
      dtype='object')

In [201]:
for column in df.columns:
    df[column] = df[column]/df[column].abs().max() # Divide by the maximum of the column which will make max value of each column is 1
df.head()

,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
Area,,,,,,,,,,
4537,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
2872,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
3048,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3073,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
3693,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [202]:
X = np.array(df.iloc[0:,0:-1])
Y = np.array(df['Class'])

In [203]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)
x_test,x_val,y_test,y_val = train_test_split(x_test,y_test,test_size=0.5,random_state=42)

print(f'Percentage of the data for training is: {round((x_train.shape[0]/X.shape[0]) *100)}')
print(f'Percentage of the data for testing is: {round((x_test.shape[0]/X.shape[0]) *100)}')
print(f'Percentage of the data for validation is: {round((x_val.shape[0]/X.shape[0]) *100)}')

Percentage of the data for training is: 70
Percentage of the data for testing is: 15
Percentage of the data for validation is: 15


In [204]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
batch_size = 32
learning_rate = 1e-3
epochs = 5

In [205]:
class DataPrep(Dataset):
    def __init__(self,X,Y):
        self.x = torch.tensor(X,dtype=torch.float32).to(device)
        self.y = torch.tensor(Y,dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.x.shape)
    
    def __getitem__(self, index):
        return self.x[index],self.y[index]

training_data = DataPrep(x_train,y_train)  
testing_data = DataPrep(x_test,y_test)
validation_data = DataPrep(x_val,y_val)


train_loader = DataLoader(training_data,shuffle=True,batch_size=batch_size)
test_loader = DataLoader(testing_data,shuffle=True,batch_size=batch_size)
val_loader = DataLoader(validation_data,shuffle=True,batch_size=batch_size)

In [206]:
class MyModel(nn.Module):
    def __init__(self,input_shape):
        super(MyModel,self).__init__()
    
        self.input = nn.Linear(input_shape,20)
        self.h1 = nn.Linear(20,10)
        self.h2 = nn.Linear(10,5)
        self.output = nn.Linear(5,1)

        self.in_activation = nn.ReLU()
        self.out_activation = nn.Sigmoid()

    def forward(self,x):
        x = self.in_activation(self.input(x))
        x = self.in_activation(self.h1(x))
        x = self.in_activation(self.h2(x))
        x = self.out_activation(self.output(x))
        return x 

In [207]:
model = MyModel(input_shape=X.shape[1]).to(device)
summary(model,input_size=(X.shape[1],))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [1]                       --
├─Linear: 1-1                            [20]                      200
├─ReLU: 1-2                              [20]                      --
├─Linear: 1-3                            [10]                      210
├─ReLU: 1-4                              [10]                      --
├─Linear: 1-5                            [5]                       55
├─ReLU: 1-6                              [5]                       --
├─Linear: 1-7                            [1]                       6
├─Sigmoid: 1-8                           [1]                       --
Total params: 471
Trainable params: 471
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [208]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(),lr=learning_rate)
model.train()

MyModel(
  (input): Linear(in_features=9, out_features=20, bias=True)
  (h1): Linear(in_features=20, out_features=10, bias=True)
  (h2): Linear(in_features=10, out_features=5, bias=True)
  (output): Linear(in_features=5, out_features=1, bias=True)
  (in_activation): ReLU()
  (out_activation): Sigmoid()
)

In [210]:

total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

for epoch in range(20):
    total_acc_train = 0
    total_loss_train = 0
    total_acc_val = 0
    total_loss_val = 0
    ## Training and Validation
    for data in train_loader:

        inputs, labels = data

        prediction = model(inputs).squeeze(1)

        batch_loss = criterion(prediction, labels)

        total_loss_train += batch_loss.item()

        acc = ((prediction).round() == labels).sum().item()

        total_acc_train += acc

        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    ## Validation
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data

            prediction = model(inputs).squeeze(1)

            batch_loss = criterion(prediction, labels)

            total_loss_val += batch_loss.item()

            acc = ((prediction).round() == labels).sum().item()

            total_acc_val += acc

    total_loss_train_plot.append(round(total_loss_train/18185, 4))
    total_loss_validation_plot.append(round(total_loss_val/18185, 4))
    total_acc_train_plot.append(round(total_acc_train/(training_data.__len__())*100, 4))
    total_acc_validation_plot.append(round(total_acc_val/(validation_data.__len__())*100, 4))

    print(f'''Epoch no. {epoch + 1} Train Loss: {total_loss_train/18185:.4f} Train Accuracy: {(total_acc_train/(training_data.__len__())*100):.4f} Validation Loss: {total_loss_val/1000:.4f} Validation Accuracy: {(total_acc_val/(validation_data.__len__())*100):.4f}''')
    print("="*50)
     

Epoch no. 1 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 2 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 3 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 4 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 5 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 6 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 7 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 8 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 9 Train Loss: 0.0000 Train Accuracy: 100.0000 Validation Loss: 0.0006 Validation Accuracy: 100.0000
Epoch no. 